In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

db_path = os.getenv("DB_OLD_PATH")
upd_path = os.getenv("DB_PATH")

Делаем так, чтобы все фильтры можно было делать по ответам

Для этого уводим MULTI -> SINGLE

In [ ]:
df = pd.read_parquet(db_path, engine="fastparquet")

# !!! Выбираем только последнюю волну, в остальных надо чекать формулировки !!!
# df = df[df["wave"] == "2025-03"]

# df - датасет со столбцами:
# - wave: category
# - respondent_id: int
# - respondent_uid: int
# - question: category
# - answer: category

In [ ]:
def _parse_formatted_question(col: pd.Series):
    """ Функция, разбивающая отформатированный вопрос на составные части """
    pattern = r"^\[(?P<tag>[^\]]+)?\]\s*(?P<q_clean>[^@|]+)(?:\s*@\s*(?P<detail>[^|]+))?(?:\s*\|\s*(?P<option>.+))?$"

    df = col.str.extract(pattern)
    df = df.apply(lambda col: col.str.strip())

    return df

def get_question_parts(
    col: pd.Series,
    out: list[str] = ['tag', 'q_clean']
):
    QS = _parse_formatted_question(col)

    if "type" in out:
        m_det = QS["detail"].notna()
        m_opt = QS["option"].notna()

        # MIX, SINGLE, MULTI
        QS.loc[m_opt, "type"] = "MULTI"
        QS.loc[m_det & ~ m_opt, "type"] = "MIX"
        QS.loc[~ m_det & ~ m_opt, "type"] = "SINGLE"

    return QS[out]

In [ ]:
q_parts = get_question_parts(df["question"].cat.categories.to_series(), ["type", "tag", "q_clean", "detail", "option"])

In [ ]:
merged_df = pd.merge(
    df, q_parts,
    left_on="question", right_index=True,
    how="left"
)

In [ ]:
m_multi = merged_df["type"] == "MULTI"
merged_df.loc[m_multi, "answer_new"] = merged_df.loc[m_multi, "option"]
merged_df.loc[~ m_multi, "answer_new"] = merged_df.loc[~ m_multi, "answer"].astype("string")

merged_df.loc[:, "question_new"] = "[" + merged_df["tag"] + "] " + merged_df["q_clean"]
m_detail = merged_df["detail"].notna()
m_option = merged_df["option"].notna()

merged_df.loc[m_detail, "question_new"] = merged_df.loc[m_detail, "question_new"] + " @ " + merged_df.loc[m_detail, "detail"]

merged_df = merged_df[["wave", "respondent_id", "respondent_uid", "question_new", "answer_new"]]\
    .rename(columns={"question_new": "question", "answer_new": "answer"})

merged_df["question"] = merged_df["question"].astype("string").astype("category")
merged_df["answer"] = merged_df["answer"].astype("string").astype("category")

In [ ]:
merged_df.to_parquet(upd_path, engine="fastparquet")